<a href="https://colab.research.google.com/github/RafsanJany-44/Thesis_Project/blob/main/SignalReading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mne

     |████████████████████████████████| 7.5 MB 8.7 MB/s 


In [4]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import ShuffleSplit, cross_val_score

from mne import Epochs, pick_types, find_events, pick_types, set_eeg_reference
from mne.channels import read_layout
from mne.io import concatenate_raws, read_raw_edf
from mne.datasets import eegbci
from mne.decoding import CSP
from mne import viz

%matplotlib inline

In [5]:
# The eeg data contains four event markers. We'll use these to explore the data.
event_id = dict(response=4096, target=8192, NonTarget_High=16384, NonTarget_Low=32768)

# This will determine the time window for the data that we care about
tmin, tmax = -0.1, 1.

# path to data (change this to whereever you saved your data)
myPath = r'/content/SN001_sleepscoring.edf'

In [ ]:
# multiple subjects

raw_fnames = eegbci.load_data(subject, runs)
raw_fnames = [os.path.join(myPath,f) for f in os.listdir(myPath)]
raw_files = [read_raw_edf(f, preload=True, stim_channel='auto', verbose=False) for f in
              raw_fnames]
raw = concatenate_raws(raw_files, verbose=False)

#2) Load our EEG data¶

In [6]:
# single subject
data = os.listdir(myPath)
dataPath = os.path.join(myPath, data[0])

# We use mne.read_raw_edf to read in the .bdf EEG files
raw = read_raw_edf(dataPath, preload=True, stim_channel='auto', verbose=False)
raw

NotADirectoryError: ignored

#Let's explore our EEG data!

In [ ]:
# what does our data look like?

print('Data type: {}\n\n{}\n'.format(type(raw), raw))
# print('{} \n'.format(raw))

# Get the sample rate
print('Sample rate:', raw.info['sfreq'], 'Hz')

# Get the size of the matrix
print('Size of the matrix: {}\n'.format(raw.get_data().shape))

# We can use the mne.info class to learn moure about the data.
print(raw.info)

In [ ]:
print('The actual data is just a matrix array!\n\n {}\n'.format(raw.get_data()))

#Start to visualize the data

In [ ]:
# Extract data from the first 5 channels, from 1 s to 3 s.
sfreq = raw.info['sfreq']
sfreq = raw.info['sfreq']
data, times = raw[:5, int(sfreq * 1):int(sfreq * 10)]

fig = plt.subplots(figsize=(10,8))
plt.plot(times, data.T);
plt.xlabel('Seconds')
plt.ylabel('$\mu V$')
plt.title('Channels: 1-5');
plt.legend(raw.ch_names[:5]);

In [ ]:
# Apply band-pass filter
raw.filter(7., 30., fir_design='firwin', skip_by_annotation='edge')

In [ ]:
data_f, times_f = raw[:5, int(sfreq * 1):int(sfreq * 10)]

fig, (ax1, ax2) = plt.subplots(1,2, figsize = (15, 10))

ax1.plot(times, data.T);
ax1.set_title('Before Filter')
ax1.set_xlabel('Seconds')
ax1.set_ylabel('$\mu V$')

ax2.plot(times_f, data_f.T);
ax2.set_title('After Filter')
ax2.set_xlabel('Seconds')

plt.legend(raw.ch_names[:5], loc=1);
# plt.tight_layout()

In [ ]:
# %matplotlib qt
raw.plot(duration=60, block=True)

In [ ]:
# %matplotlib qt
raw.plot_psd(tmax=np.inf, fmax=250)

In [ ]:
raw_df = raw.to_data_frame()
raw_df.head()

#3) Epochs:


In [ ]:
%matplotlib inline
picks = pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False,
                   exclude='bads')

# events.shap|e
events = find_events(raw, shortest_event=0, stim_channel='STI 014', verbose=False)

epochs = Epochs(raw, events, event_id, tmin, tmax, proj=True, picks=picks,
                baseline=None, preload=True, verbose=False)

# remove channels that aren't eeg electrodes we care about
epochs.drop_channels(ch_names=['Nose', 'REOG', 'LEOG', 'IEOG', 'SEOG', 'M1', 'M2','EXG8'])

# Export data in tabular structure as a pandas DataFrame.
epochs_df = epochs.to_data_frame()


evoked = epochs['target'].average()
evoked.plot();

In [ ]:
evoked_no_ref = mne.Epochs(raw_no_ref, **epochs_params).average()
evoked_no_ref.drop_channels(ch_names=['Nose', 'REOG', 'LEOG', 'IEOG', 'SEOG', 'M1', 'M2','EXG8'])

# del raw_no_ref  # save memory

title = 'EEG Original reference'
evoked_no_ref.plot(titles=dict(eeg=title), time_unit='s')
# evoked_no_ref.plot_topomap(times=[0.1], size=3., title=title, time_unit='s')

In [ ]:
picks = pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False,
                   exclude='bads')

# events.shap|e
events = find_events(raw, shortest_event=0, stim_channel='STI 014')

epochs = Epochs(raw, events, event_id, tmin, tmax, proj=True, picks=picks,
                baseline=None, preload=True)

# remove channels that aren't eeg electrodes we care about
epochs.drop_channels(ch_names=['Nose', 'REOG', 'LEOG', 'IEOG', 'SEOG', 'M1', 'M2','EXG8', 'P1', 'PO7'])

# Export data in tabular structure as a pandas DataFrame.
epochs_df = epochs.to_data_frame()


evoked = epochs['target'].average()
evoked.plot()

In [ ]:
df=pd.DataFrame(events, columns = ['time','x','event'])
df.index = df['time']/1024

fig, ax = plt.subplots()

ax.scatter(df.index, df['event'])

indexA = np.array(df.index[df['event']==8192])
for i in indexA:
    ax.axvline(x=i, ymin=0, ymax=1, c='r', linestyle='--')

In [ ]:
%matplotlib inline

viz.plot_events(events, raw.info['sfreq'], raw.first_samp,
                    event_id=event_id)

plt.show()


In [ ]:
epochs['target'].get_data().shape
plt.plot(epochs['target'].get_data()[:,26,:][0])

In [ ]:
import mne
picks_x = mne.pick_channels(raw.ch_names, include=['O1'])
# p_right = mne.pick_channels(raw.ch_names, ['O2'])

epochs_x = Epochs(raw, events, event_id, tmin, tmax, proj=True, picks=picks_x,
                baseline=None, preload=True)


evoked_x = epochs_x['target'].average()
evoked_x.plot(time_unit='s')


epochs_dfx = epochs_x.to_data_frame()

# epochs_df.index
x = epochs_dfx.xs(('target'), level=0)
# x.xs((0), level=0)
# x.groupby(['epoch', 'time']).mean()
data_mean = x.mean(level='time')

fig, ax = plt.subplots(figsize=(20,10))

ax.plot(data_mean['O1'])
print(ax.get_ylim())
plt.vlines(0, ymin=ax.get_ylim()[0], ymax=ax.get_ylim()[1], linestyle='--')
# x.loc[0]['O1'].plot(figsize=(15,6))

plt.legend(['O1','O2'])

plt.show()

In [ ]:
%matplotlib qt
epochs.plot(block=True)
plt.show()

In [ ]:
epochs['target'].plot_image(combine=None, cmap='interactive')

In [ ]:
raw.plot_psd(tmax=np.inf, average=False)

In [ ]:
targets = [x[0] for x in events if x[2]==8192]
# times = np.array(times)

In [ ]:
df = pd.DataFrame(events, columns=['time','num','event'])

In [ ]:
event_id['target']

In [ ]:
df['event'].value_counts()

In [ ]:
data, time = raw[:]

In [ ]:
# make a EEG DataFrame 

millis = (1/1024) * 1000
times = np.arange(0,len(raw)-5, millis)
times = [x + millis for x in np.arange(len(raw))]

eeg = pd.DataFrame(data.transpose(), columns=raw.ch_names, index=times)
eeg = eeg.iloc[:,:64]

In [ ]:
np.arange(len(axs))

In [ ]:
np.array(time[0:97]).diff